#### 입력 : ./make_file/division/third_round/서울시_도로_{숫자}.csv
#### 출력 : ./make_file/(통합)_서울시_도로_리스트.csv

In [1]:
import pandas as pd 
import requests
from tqdm import tqdm 
import os
import time

In [2]:
# 파일이 저장된 폴더 경로
folder_path = "./make_file/division/third_round/" 
file_encoding = "UTF-8"

In [3]:
# 조건을 만족하는 파일 목록
valid_files = []

for temp_file_name in os.listdir(folder_path):
    file_name = temp_file_name.replace("_", " ")
    # file_name = temp_file_name
    if ("서울시 도로 " in file_name and file_name.endswith(".csv")) or ("서울시 도로 " in file_name and file_name.endswith(".csv")):
        file_path = os.path.join(folder_path, temp_file_name)  # 안전한 경로 조합

        try:
            # CSV 파일 전체 읽기
            df = pd.read_csv(file_path, encoding=file_encoding, low_memory=False)

            # 컬럼명 앞뒤 공백 제거
            df.columns = df.columns.str.strip()

            # 필요한 컬럼이 존재하는지 확인
            if {"중앙점_위도", "중앙점_경도", "행정동"}.issubset(df.columns):
                valid_files.append(file_path)
        except Exception as e:
            print(f"파일 {file_name} 처리 중 오류 발생: {e}")
    else :
        print(file_name)
        

.DS Store


In [4]:
valid_files

['./make_file/division/third_round/보정_서울시_도로_01.csv',
 './make_file/division/third_round/보정_서울시_도로_15.csv',
 './make_file/division/third_round/보정_서울시_도로_14.csv',
 './make_file/division/third_round/보정_서울시_도로_16.csv',
 './make_file/division/third_round/보정_서울시_도로_02.csv',
 './make_file/division/third_round/보정_서울시_도로_03.csv',
 './make_file/division/third_round/보정_서울시_도로_17.csv',
 './make_file/division/third_round/보정_서울시_도로_13.csv',
 './make_file/division/third_round/보정_서울시_도로_07.csv',
 './make_file/division/third_round/보정_서울시_도로_06.csv',
 './make_file/division/third_round/보정_서울시_도로_12.csv',
 './make_file/division/third_round/보정_서울시_도로_04.csv',
 './make_file/division/third_round/보정_서울시_도로_10.csv',
 './make_file/division/third_round/보정_서울시_도로_11.csv',
 './make_file/division/third_round/보정_서울시_도로_05.csv',
 './make_file/division/third_round/보정_서울시_도로_08.csv',
 './make_file/division/third_round/보정_서울시_도로_09.csv']

In [5]:
len(valid_files)

17

In [6]:
# 합친 데이터를 저장할 빈 데이터프레임
combined_df = pd.DataFrame()

# valid_files에 있는 모든 파일을 하나의 데이터프레임으로 합침
for file_path in valid_files:
    try:
        # CSV 파일 읽기
        df = pd.read_csv(file_path, encoding=file_encoding, low_memory=False)

        # 데이터를 하나로 합치기
        combined_df = pd.concat([combined_df, df], ignore_index=True)
        
    except Exception as e:
        print(f"파일 {file_path} 처리 중 오류 발생: {e}")

In [7]:
# for i in tqdm(range(len(combined_df))):
#     if pd.isna(combined_df['행정동'][i]) or combined_df['행정동'][i] == "nan":
#         print(combined_df['도로명'][i], combined_df['중앙점_위도'][i], combined_df['중앙점_경도'][i])

In [8]:
nan_count = combined_df['행정동'].isna().sum() + (combined_df['행정동'] == "nan").sum()
print(nan_count)

283


In [9]:
# 널 값 삭제 후 결과 확인
combined_df = combined_df[~combined_df['행정동'].isna()]
nan_count = combined_df['행정동'].isna().sum() + (combined_df['행정동'] == "nan").sum()
print(nan_count)

# 'nan' 값 삭제 후 결과 확인
combined_df = combined_df[combined_df['행정동'] != "nan"]
nan_count = combined_df['행정동'].isna().sum() + (combined_df['행정동'] == "nan").sum()
print(nan_count)


0
0


In [10]:
combined_df.columns

Index(['도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '터널 여부', '교량 여부', '서비스 도로 여부', '도로 ID', '속도 제한',
       '차선 수', '일방통행 여부', '교차로 여부', '행정동'],
      dtype='object')

In [11]:
combined_df['열선'] = 0

In [12]:
combined_df.columns

Index(['도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '터널 여부', '교량 여부', '서비스 도로 여부', '도로 ID', '속도 제한',
       '차선 수', '일방통행 여부', '교차로 여부', '행정동', '열선'],
      dtype='object')

In [13]:
new_combined_df = combined_df[['도로명', '열선', '도로 종류', '시작점_위도', '시작점_경도', 
                               '종료점_위도', '종료점_경도', '중앙점_위도', '중앙점_경도', 
                               '도로_길이', '행정동']].copy()


In [14]:
print(f"저장하는 도로의 수 : {len(new_combined_df):,}곳")

저장하는 도로의 수 : 161,362곳


In [15]:
try :
    save_url = "./make_file/(통합)_서울시_도로_리스트.csv"
    new_combined_df.to_csv(save_url, index=False, encoding='UTF-8')
    print(f"{save_url} 파일 저장 완료.")
except OSError as e :
    print(e)

./make_file/(통합)_서울시_도로_리스트.csv 파일 저장 완료.
